Django提高01
====
**玩转context对象**

https://docs.djangoproject.com/en/2.0/ref/templates/api/#playing-with-context

大多数情况下，通过将完全填充的字典传递给Context()来实例化Context对象。

在字典被实例化为Context对象后，仍然可以使用标准字典语法对实例化后Context对象后添加和删除它们：

```
>>> from django.template import Context
>>> c = Context({"foo": "bar"})
>>> c['foo']
'bar'
>>> del c['foo']
>>> c['foo']
Traceback (most recent call last):
...
KeyError: 'foo'
>>> c['newvariable'] = 'hello'
>>> c['newvariable']
'hello'
```


# Context.get(key, otherwise=None)

Context对象的 get 方法，如果 key 在 Context 中，就返回 key 对应的值；否则返回 otherwise 参数值。

```
python manage.py shell

from django.template import Context

c = Context({'art_tag':'Django','author':'Anguo','last_time':'20180601'})

c.get('art_tag','无此键')  #otherwise可以是一个字符串，也可以是一个变量

In [7]: c.get('art_tag','无此键')
Out[7]: 'Django'

In [8]: c.get('author','无此键')
Out[8]: 'Anguo'

In [9]: c.get('title','无此键')
Out[9]: '无此键'

In [10]: c.get('last_time','无此键')
Out[10]: '20180601'
```

# Context.setdefault(key, default=None)

如果Context对象中有这个 key，就返回它的值。否则就插入这个key，并映射 default 值，再返回 default 值。

```
In [65]: c=Context()

In [66]: c['keya']='aaa'

In [67]: c['keyb']='bbb'

In [68]: c
Out[68]: [{'True': True, 'False': False, 'None': None, 'keya': 'aaa', 'keyb': 'bbb'}]

In [69]: c.setdefault('keyd','ddd')
Out[69]: 'ddd'

In [70]: c
Out[70]: [{'True': True, 'False': False, 'None': None, 'keya': 'aaa', 'keyb': 'bbb', 'keyd': 'ddd'}]

In [71]: c.setdefault('keya','ddd')
Out[71]: 'aaa'
```

Context对象是一个 Stack（堆栈），实现了一个先进后出的数据结构。它有两个方法:

- Context.pop()，移除栈顶（序列的最后一个）元素，pop, 取出。
- Context.push()，在栈顶（序列的最后位置）增加元素，push, 增加。

当pop()次数比push()次数多时就会抛出ContextPopException异常。

```
class ContextPopException(Exception):
    "pop() has been called more times than push()"
    pass
```

可以使用 push() 作为上下文管理器来确保调用匹配的 pop()。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> with c.push():
...     c['foo'] = 'second level'
...     c['foo']
'second level'
>>> c['foo']
'first level'
```

传递给 push() 的所有参数都将传递给dict构造函数用于构建新一级上下文。

```
In [18]: c=Context()

In [19]: c
Out[19]: [{'True': True, 'False': False, 'None': None}]

In [20]: c['mik']='first level'

In [21]: c
Out[21]: [{'True': True, 'False': False, 'None': None, 'mik': 'first level'}]

In [31]: with c.push(mik='second level'):
    ...:     print(c)
    ...:
[{'True': True, 'False': False, 'None': None, 'mik': 'first level'}, {'mik': 'second level'}]

In [32]: c
Out[32]: [{'True': True, 'False': False, 'None': None, 'mik': 'first level'}]

```

# Context.update(other_dict)

除了push（）和pop（）之外，Context对象还定义了update（）方法。 这与push（）类似，但是将字典作为参数，并将该字典推送到堆栈而不是空字典。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> c.update({'foo': 'updated'})
{'foo': 'updated'}
>>> c['foo']
'updated'
>>> c.pop()
{'foo': 'updated'}
>>> c['foo']
'first level'
```

与 push() 一样，您可以使用 update() 作为上下文管理器来确保调用匹配的 pop()。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> with c.update({'foo': 'second level'}):
...     c['foo']
'second level'
>>> c['foo']
'first level'
```



在一些自定义模板标记中，使用上下文作为堆栈非常方便。

# Context.flatten()

使用flatten（）方法，可以将整个Context堆栈作为一个字典，包括内置变量。

```
>>> c = Context()
>>> c['foo'] = 'first level'
>>> c.update({'bar': 'second level'})
{'bar': 'second level'}
>>> c.flatten()
{'True': True, 'None': None, 'foo': 'first level', 'False': False, 'bar': 'second level'}
```

flatten（）方法也在内部用于使Context对象具有可比性。

```
>>> c1 = Context()
>>> c1['foo'] = 'first level'
>>> c1['bar'] = 'second level'
>>> c2 = Context()
>>> c2.update({'bar': 'second level', 'foo': 'first level'})
{'foo': 'first level', 'bar': 'second level'}
>>> c1 == c2
True
```

flatten（）的结果在单元测试中可用于比较 Context 与 dict：

```
class ContextTest(unittest.TestCase):
    def test_against_dictionary(self):
        c1 = Context()
        c1['update'] = 'value'
        self.assertEqual(c1.flatten(), {
            'True': True,
            'None': None,
            'False': False,
            'update': 'value',
        })
```